In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pptx import Presentation
from datetime import datetime

# Define the root directory
root_dir = r'U:\NVL\HX\A0\results_experimental\ThermalProfile'

# Dictionary to store data for each subfolder
subfolder_data = {}

# Function to process each file and store data by subfolder
def process_file(file_path, subfolder_name):
    try:
        # Skip temporary files (starting with ~) without printing a message
        if os.path.basename(file_path).startswith('~$'):
            return
        
        # Use pd.read_excel directly instead of ExcelFile
        df = pd.read_excel(file_path, sheet_name='SearchVoltage Results', engine='openpyxl')
        required_cols = ['run_order', 'cmvprofiling.Max_DTS_Profile_max', 
                       'cmvprofiling.Intec_TC_Profile_max', 'cmvprofiling.Intec_FB_Profile_max']
        if all(col in df.columns for col in required_cols):
            # Filter rows where all required columns have non-null values
            valid_data = df[required_cols].dropna()
            
            # Initialize subfolder data if not present
            if subfolder_name not in subfolder_data:
                subfolder_data[subfolder_name] = {'run_x': [], 'dts_y': [], 'tc_y': [], 'fb_y': []}
            
            # Convert to integers with robust handling
            run_x_data = valid_data['run_order'].tolist()
            dts_y_data = pd.to_numeric(valid_data['cmvprofiling.Max_DTS_Profile_max'], errors='coerce').dropna().astype(int).tolist()
            tc_y_data = pd.to_numeric(valid_data['cmvprofiling.Intec_TC_Profile_max'], errors='coerce').dropna().astype(int).tolist()
            fb_y_data = pd.to_numeric(valid_data['cmvprofiling.Intec_FB_Profile_max'], errors='coerce').dropna().astype(int).tolist()
            
            # Debug: Print lengths to check for mismatches
            print(f"File: {file_path}")
            print(f"Length of run_x: {len(run_x_data)}, dts_y: {len(dts_y_data)}, tc_y: {len(tc_y_data)}, fb_y: {len(fb_y_data)}")
            
            # Find the minimum length to trim all lists
            min_length = min(len(run_x_data), len(dts_y_data), len(tc_y_data), len(fb_y_data))
            if min_length > 0:
                subfolder_data[subfolder_name]['run_x'].extend(run_x_data[:min_length])
                subfolder_data[subfolder_name]['dts_y'].extend(dts_y_data[:min_length])
                subfolder_data[subfolder_name]['tc_y'].extend(tc_y_data[:min_length])
                subfolder_data[subfolder_name]['fb_y'].extend(fb_y_data[:min_length])
            else:
                print(f"Warning: No valid data after processing {file_path}.")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Traverse the directory structure
try:
    main_folders = [f for f in os.listdir(root_dir) if 'D3' in f or 'D4' in f]
    for main in main_folders:
        main_path = os.path.join(root_dir, main)
        # Reset subfolder_data for each main folder to avoid cross-contamination
        subfolder_data.clear()
        
        # Debug: Confirm processing of each main folder
        print(f"Processing main folder: {main}")
        
        # Get immediate subfolders under the main folder
        subfolders = [d for d in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, d))]
        for subfolder in subfolders:
            subfolder_path = os.path.join(main_path, subfolder)
            for dirpath, dirnames, filenames in os.walk(subfolder_path):
                # Use the subfolder name for data separation
                subfolder_name = subfolder
                for filename in filenames:
                    if filename.endswith('.xlsx') and ('HotVmin' in filename or 'HotGNG' in filename):
                        file_path = os.path.join(dirpath, filename)
                        process_file(file_path, subfolder_name)

        # Create a PowerPoint presentation for the current main folder
        prs = Presentation()
        slide_layout = prs.slide_layouts[5]  # Use a layout with a title and content (e.g., layout 5)

        # Create a separate plot and slide for each subfolder
        for subfolder, data in subfolder_data.items():
            if any(data.values()):  # Create plot if any list has data
                # Create a figure
                plt.figure(figsize=(18, 8))
                plt.scatter(data['run_x'], data['dts_y'], color='blue', label='cmvprofiling.Max_DTS_Profile_max')
                plt.scatter(data['run_x'], data['tc_y'], color='green', label='cmvprofiling.Intec_TC_Profile_max')
                plt.scatter(data['run_x'], data['fb_y'], color='red', label='cmvprofiling.Intec_FB_Profile_max')

                plt.title(f'Thermal Profiling - {subfolder}')
                plt.xlabel('run_order')
                plt.ylabel('Temperature')
                plt.legend(loc='upper right', fontsize=8)
                plt.grid(True)
                plt.tight_layout()

                # Save the plot as an image
                image_path = f'temp_plot_{subfolder}.png'
                plt.savefig(image_path, bbox_inches='tight', dpi=300)
                plt.close()  # Close the figure to free memory

                # Add a slide to the PowerPoint
                slide = prs.slides.add_slide(slide_layout)
                title = slide.shapes.title
                title.text = f'Thermal Profiling - {subfolder}'
                pic = slide.shapes.add_picture(image_path, 50, 100, width=prs.slide_width - 100)  # Adjust position and size

                # Remove the temporary image file
                os.remove(image_path)

        # Define the custom save path for the PowerPoint file with timestamp and main folder name
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')  # Format: YYYYMMDD_HHMMSS
        output_pptx = r'U:\NVL\HX\A0\results_experimental\ThermalProfileSummary\Thermal_Profiling_Results_{}_{}.pptx'.format(main, timestamp)
        # Ensure the output directory exists
        os.makedirs(os.path.dirname(output_pptx), exist_ok=True)

        # Save the PowerPoint file for the current main folder
        prs.save(output_pptx)
        print(f"PowerPoint file saved as {output_pptx}")

        # Ensure the script ends properly
        print("Script execution completed.")
        
except Exception as e:
        print(f"Error processing {file_path}: {e}")